#Assignment description

**Prerequisites**:

1. Authorize with your Google account
2. Go to Google Cloud Console
3. Select existing Google Cloud project or create a new one
4. Enable Google BigQuery API for the selected project
5. Copy this document to your google drive

**Implementation**:
1. Complete all assignments in this document
2. Share your variant of the document with us

In [1]:
#run this cell first
import numpy as np
import pandas as pd
from google.colab import auth
from operator import mul
from functools import reduce
auth.authenticate_user()
print('Authenticated')

Authenticated


#Assignment #1
Prepare SQL which converts raw hits (user events) into sessions.

Count the number of sessions in the result ta    ble.

**In**:
 - [Session definition](https://support.google.com/analytics/answer/2731565?hl=en)
 - Public table with samle hits data: `analytics-230012.Assignments.hits_for_sessions`
 - Use your Google Cloud project ID instead of  `project_id`

In [33]:
query = '''
#Start your query here


  SELECT SUM (is_new_session) AS session_count
    FROM (
      SELECT CASE WHEN last_event IS NULL THEN 1 ELSE 0 END AS is_new_session
        FROM (
            SELECT *,
                   LAG(hitTime,1) OVER
                      (PARTITION BY anonymousId ORDER BY hitTime)
                      AS last_event
              FROM `analytics-230012.Assignments.hits_for_sessions`
           ) last
          )


#Finish your query here
'''

project_id = 'moonlit-rock-408909' #change

df = pd.io.gbq.read_gbq(query, project_id=project_id, dialect='standard')
df

,session_count
0,545
























#Assignment #2
Count all the users matching the following criteria:
- Have multiple transactions
- Have at least one transaction that was made within 7 days from the previous transaction

**In**:
 - Public table with samle transactions data: `analytics-230012.Assignments.orders`
 - Use your Google Cloud project ID instead of  `project_id`

In [41]:
query = '''
#Start your query here

WITH intermediate_table AS (
    SELECT
      userId,
      date,
      (LAG(date) OVER (PARTITION BY userId ORDER BY date)) AS previous_order
  FROM `analytics-230012.Assignments.orders`
)

SELECT userId
FROM intermediate_table
WHERE DATE_DIFF(date, previous_order, DAY) <= 7
GROUP BY userId


#Finish your query here
'''

project_id = 'moonlit-rock-408909' #change

df = pd.io.gbq.read_gbq(query, project_id=project_id, dialect='standard')
df

,userId
0,125845
1,2577
2,39093
3,4128
4,6132
...,...
1619,107362
1620,13720
1621,21232
1622,35404


#Assignment #3
Below you can see a matrix. We highlighted few sequences of 5 neighbour numbers in different directions. There are 4 directions: vertical, horizontal, diagonal1(left,down -> right,up), diagonal2(left,up -> right,down).
 - Product of 5 horizontal numbers example: 38 x 88 x 57 x 19 x 22 = 79674144
 - Product of 5 diagonal1 numbers example: 84 x 58 x 83 x 10 x 90 = 363938400

**What is the maximum product for any sequence?**

![alt text](https://drive.google.com/uc?export=view&id=1TGORTF_ffAeu5ujS5s461U2kkzUErN3w)

In [ ]:
def largestProduct(matrix):
  maxProduct = 0
  #start your code here

  if(type(matrix) == str):
    matrix = np.fromstring(matrix, dtype=int, sep=' ')
    matrix = matrix.reshape(int(np.sqrt(len(matrix))), int(np.sqrt(len(matrix))))

  for i in range(len(matrix[0]) - 5):
    for j in range(len(matrix[0]) - 5):
      # комбинации по стобцам
      maxProduct = max(maxProduct, reduce(mul, matrix[i:i+5, j] if i+5 < len(matrix[0]) else [maxProduct], 1))

      # комбинации по строкам
      maxProduct = max(maxProduct, reduce(mul, matrix[i][j:j+5] if j+5 < len(matrix[0]) else [maxProduct], 1))


      # комбинации по диагоналям left,up -> right,down
      maxProduct = max(maxProduct, reduce(mul, [matrix[i+ix, j+ix] for ix in range(5)] if j+5 < len(matrix[0]) and i + 5 < len(matrix[0]) else [maxProduct], 1))

      # комбинации по диагоналям left,down -> right,up
      maxProduct = max(maxProduct, reduce(mul, [matrix[i+ix, j - ix + 5] for ix in range(5)] if j+5 < len(matrix[0]) and i + 5 < len(matrix[0]) else [maxProduct], 1))



  #finish your code here
  return maxProduct

matrix = '''30 90 80 36 44 66 42 34 65 52 88 26 79 81 55 92 37 37 04 84
93 04 92 03 37 87 93 84 94 62 55 07 88 86 09 33 70 78 09 61
72 02 02 37 38 88 57 19 22 82 71 53 14 60 73 05 45 77 76 57
32 34 77 55 39 93 36 95 65 32 95 90 53 71 97 70 85 38 72 37
37 53 33 97 41 01 77 07 68 10 01 45 21 94 09 80 98 45 89 87
19 12 58 02 27 83 70 12 75 12 36 43 15 79 40 90 91 74 58 84
45 34 34 29 14 89 51 65 87 24 78 26 94 82 64 75 14 09 76 26
86 30 86 21 30 00 57 90 11 00 41 48 08 38 37 41 85 84 46 74
61 83 37 84 98 44 10 32 08 18 68 36 43 49 26 28 47 37 81 23
39 74 01 51 73 83 85 41 82 25 03 32 16 15 66 51 37 68 47 28
78 68 27 37 58 88 63 54 89 11 56 47 20 57 58 07 07 36 26 20
11 36 87 84 82 68 12 16 37 16 00 50 96 25 53 00 27 78 00 57
04 70 06 96 92 00 96 15 24 32 13 49 61 52 30 38 03 77 43 17
45 78 70 45 10 61 60 71 00 18 91 27 00 04 66 72 07 74 89 73
94 97 29 25 89 67 91 65 17 27 51 61 86 21 95 79 01 54 89 80
59 90 72 94 60 03 39 61 69 49 52 89 50 74 84 56 50 86 68 26
66 47 67 78 68 65 20 83 78 37 19 52 16 76 12 25 35 81 26 81
67 03 31 26 27 61 36 84 19 53 77 69 40 13 91 78 00 54 94 02
20 59 70 91 74 39 90 55 53 74 12 79 48 40 01 73 06 68 56 40
07 33 59 64 84 25 89 38 19 29 93 77 46 86 45 93 63 96 53 29'''
print(largestProduct(matrix))

4703677440
